# Resample hourly to daily

This notebook is used for resampling the restacked 20km outputs to daily files. 

We will submit a slurm job for a subset of the hourly files, that will then be resampled to a daily resolution.

In [1]:
import xarray as xr
from config import *
import luts
import slurm
from restack_20km import user_input_years, user_input_variables, get_year_fn_str

Specify the years and variables to be resampled:

In [2]:
years = user_input_years()
varnames = user_input_variables(luts.resample_varnames)

Years, a ' '- separated list of years, or '-'-separated start and end year (e.g. 2005-2010) for a range of years.
Years must be in 2005-2100. Leave blank for all years: 2064-2100


Years selected: [2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100]


Enter name(s) of WRF variable(s) to restack (leave blank for all): 


Iterate over the output variable names, i.e. the resampled/aggregated variable names, which may be the same as the WRF variable name, and write the sbatch scripts:

In [3]:
year_str = get_year_fn_str(years)

sbatch_dir = slurm_dir.joinpath("resample_daily")
sbatch_dir.mkdir(exist_ok=True)
sbatch_fps = []
for varname in varnames:
    varname = varname.lower()
    wrf_varname = luts.resample_varnames[varname.upper()]["wrf_varname"]
    # write to .slurm script
    sbatch_fp = sbatch_dir.joinpath(f"resample_{group}_{year_str}_{varname}.slurm")
    # filepath for slurm stdout
    sbatch_out_fp = sbatch_dir.joinpath(f"resample_{group}_{year_str}_{varname}_%j.out")
    sbatch_head = slurm.make_sbatch_head(
        slurm_email, partition, conda_init_script
    )

    args = {
        "sbatch_fp": sbatch_fp,
        "sbatch_out_fp": sbatch_out_fp,
        "resample_script": resample_script,
        "hourly_dir": hourly_dir,
        "daily_dir": daily_dir,
        "wrf_varname": wrf_varname,
        "out_varname": varname,
        "aggr": luts.resample_varnames[varname.upper()]["aggr"],
        "fn_str": luts.groups[group]["fn_str"],
        "year_str": year_str,
        "ncpus": ncpus,
        "sbatch_head": sbatch_head,
    }

    slurm.write_sbatch_resample(**args)
    sbatch_fps.append(sbatch_fp)

Remove existing slurm output scripts if you fancy it:

In [4]:
for varname in varnames:
    _ = [fp.unlink() for fp in list(sbatch_dir.glob(f"resample_{group}_{year_str}_{varname}_*.out"))]

Submit the `.slurm` scripts with `sbatch`:

In [5]:
job_ids = [slurm.submit_sbatch(fp) for fp in sbatch_fps]

Once the slurm jobs have all finished, proceed to the quality control notebook. 

In [11]:
!squeue -u kmredilla

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            470735   t1small resample kmredill  R      24:10      1 n23


In [13]:
try:
    assert True == False
except AssertionError:
    print("yes")

yes
